### Matrix transpose

### 4x4

$
\begin{pmatrix}
  1 & 1 & 1 \\
  2 & 2 & 2 \\
  3 & 3 & 3
\end{pmatrix}
$

In [210]:


%%writefile matrix_transpose_row_wise.cu

#include<stdio.h>
#include<stdlib.h>

void print_2d_matrix(int *a, int r, int c) {
  for (int i = 0; i < r; i++) {
    for (int j = 0; j < c; j++) {
      printf("%d ", a[i*c+j]);
    }
    printf("\n");
  }
}

__global__ void transpose_row_wise_per_thread(int *a, int *out, int r, int c) {
  // Extract threadid
  int idx = blockIdx.x * blockDim.x + threadIdx.x;

  for (int i=0; i < c; i++) {
    out[i*c+idx] = a[idx*c+i];
  }
}

int main() {
  int R = 100;
  int C = 100;
  int *a, *d_a, *d_out, *out;

  a = (int *)malloc(sizeof(int) * R * C);
  out = (int *)malloc(sizeof(int) * R * C);

  for (int i = 0; i < R; i++) {
    for (int j = 0; j < C; j++) {
      a[(i*C) + j] = (100^i)*(10^j);
    }
  }

  // Move the memory to GPU

  cudaMalloc((void **)&d_a, R * C * sizeof(int));
  cudaMalloc((void **)&d_out, R * C * sizeof(int));

  cudaMemcpy(d_a, a, R * C * sizeof(int), cudaMemcpyHostToDevice);

  printf("Assign 2d memory col memory for the GPU pointers.\n");

  transpose_row_wise_per_thread<<<1,R>>>(d_a, d_out, R, C);

  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  cudaMemcpy(out, d_out, R * C * sizeof(int), cudaMemcpyDeviceToHost);

  printf("Out from GPU .. \n");


  for (int i=0; i<R; i++){
    for (int j =0; j <C; j++) {
      if (a[i*C+j] != out[j*C+i]) {
        printf("Incorrect transpose !!!\n");
        print_2d_matrix(out, R, C);
        print_2d_matrix(a, R, C);
      }
    }
  }

  cudaFree(d_a);
  cudaFree(d_out);

  free(out);
  free(a);

  return 0;
}

Overwriting matrix_transpose_row_wise.cu


In [211]:
!nvcc matrix_transpose_row_wise.cu  -o matrix_transpose_row_wise
!./matrix_transpose_row_wise

Assign 2d memory col memory for the GPU pointers.
Out from GPU .. 


In [212]:
!nvprof ./matrix_transpose_row_wise

==26248== NVPROF is profiling process 26248, command: ./matrix_transpose_row_wise
Assign 2d memory col memory for the GPU pointers.
Out from GPU .. 
==26248== Profiling application: ./matrix_transpose_row_wise
==26248== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.29%  21.312us         1  21.312us  21.312us  21.312us  transpose_row_wise_per_thread(int*, int*, int, int)
                   19.12%  6.2400us         1  6.2400us  6.2400us  6.2400us  [CUDA memcpy HtoD]
                   15.59%  5.0880us         1  5.0880us  5.0880us  5.0880us  [CUDA memcpy DtoH]
      API calls:   99.36%  90.928ms         2  45.464ms  7.3740us  90.921ms  cudaMalloc
                    0.23%  214.26us         1  214.26us  214.26us  214.26us  cudaLaunchKernel
                    0.15%  139.48us       114  1.2230us     147ns  55.672us  cuDeviceGetAttribute
                    0.11%  103.04us         2  51.521us  38.907us  64.135us  cu